**Installs and Imports**

In [ ]:
%pip install mne
%pip install matplotlib
%pip install pymatreader
%pip install scipy

In [ ]:
%pip install FOOOF

In [ ]:
%pip install neurodsp

In [ ]:
%pip install python-dotenv

In [ ]:
pip install loess

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

import numpy as np
import matplotlib
import mne
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa

from neurodsp.spectral import compute_spectrum, trim_spectrum


# Import NeuroDSP plotting functions
from neurodsp.plts import (plot_time_series, plot_power_spectra, plot_bursts, plot_lagged_coherence)

from fooof import FOOOF
import os
import pandas as pd
import csv

import scipy
from scipy import stats
import statsmodels.api as sm

In [ ]:
load_dotenv('PipelineVariables.env')
EEG_Directory=os.getenv('EEG_Directory')
EEG_Results_Directory=os.getenv('EEG_Results_Directory')

# **The Following functions are functions for pre-processing the data**

In [ ]:
%cd {EEG_Directory}

In [ ]:
def segment_channels(raw): ## segments the data into desired channels
  channels_to_keep = ['E1', 'E2', 'E3', 'E4', 'E8', 'E9', 'E10', 'E15', 'E16', 'E18', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E32', 'E33', 'E122', 'E123', 'E124']
  return raw.pick_channels(channels_to_keep)

In [ ]:
def avgChannels(raw): ## averages channels into one evoked object of the data for the frontal lobe
  events = mne.find_events(raw, stim_channel = None)
  epochs = mne.Epochs(raw, events, event_id=None, tmin=0, tmax=10)
  evoked = epochs.average()
  return evoked

In [ ]:
def computeFOOOF(raw):  ## FOOOF computations
  data, times = raw[:,:]
  freqs, powers = compute_spectrum(data[0], fs, method='welch', avg_type='median')
  freqs, powers = trim_spectrum(freqs, powers, [4, 28])
  return fm.report(freqs, powers, [4,28])

# **These blocks create necessary global variables for preprocessing, set our working directory, creating the FOOOF report, renaming files, and creating the PANDAS dataframe**

In [ ]:

groups=dict(frontal = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
fs = 250

In [ ]:
#PANDAS Lists to be appended then made into Data frame
PersonalID = []
age = []
sex =[]

Aper_exp = []
Aper_exp = []
variance = []
error = []

ProcessedDataDict = {'ID': PersonalID, 'Age': age, 'Sex': sex,
'Aper_offset': Aper_offset, 'Aper_exp': Aper_exp, 'Variance': variance, 'Error': error}

PeakID = []
PeakIndex = []
CenterFrequency = []
PeakWidth = []
BandWidth = []

ProcessedPeaksDataDict = {'ID': PeakID, 'Age': age, 'Sex': sex, 'Peak_Number': PeakIndex , 'CF': CenterFrequency, 'PW': PeakWidth, 'BW': BandWidth}

failedIDs = []
redundantIDs = []
otherError = []

In [ ]:
%cd {EEG_Directory}

Completed so don't run once renamed

In [ ]:
for filename in os.listdir():
  if filename.endswith('.fdt'):
    os.rename(filename, filename[::-1][22::][::-1] + '.fdt')  #renames the files
  elif filename.endswith('.set'):
    os.rename(filename, filename[::-1][22::][::-1] + '.set')
print('successfully completed')

# **Loop for preprocessing the files, creating FOOOF report, and saving the report to pandas.**

In [ ]:
phenoDF = pd.read_csv('CorrectPhenotypesRaw.csv')  #phenotpypic data frame
phenoDF = phenoDF.set_index('EID')

fm = FOOOF()

Cell runs the analysis and saves values to be made into data frame

In [ ]:
for IDs, row in phenoDF.iterrows():
  if IDs not in PersonalID:
    try:
      raw = mne.io.read_raw_eeglab(input_fname = EEG_Directory + '\sub-' + IDs + '.set') #read the EEG file that was renamed earlier in format ID.set
      raw = segment_channels(raw)
      raw = mne.channels.combine_channels(raw, groups, method = 'median')  #channel averaging
      report = computeFOOOF(raw) #FOOOF object to break apart and save in pandas df



      ## Saving results to Lists to be made into data frame ##

      #from CSV

      ProcessedDataDict['ID'].append(IDs)
      ProcessedDataDict['Age'].append(phenoDF.loc[IDs,'Age'])
      ProcessedDataDict['Sex'].append(phenoDF.loc[IDs,'Sex'])
      
      #from FOOOF
      ProcessedDataDict['Aper_offset'].append(fm.aperiodic_params_[0])
      ProcessedDataDict['Aper_exp'].append(fm.aperiodic_params_[-1])
      ProcessedDataDict['Variance'].append(fm.r_squared_)
      ProcessedDataDict['Error'].append(fm.error_)

      PeakNum = 0 
      for peak in fm.peak_params_:  #appends the IDs and peak parameters into the lists
        ProcessedPeaksDataDict['ID'].append(IDs)
        ProcessedPeaksDataDict['Peak_Number'].append(PeakNum)
        ProcessedPeaksDataDict['CF'].append(peak[0])
        ProcessedPeaksDataDict['PW'].append(peak[1])
        ProcessedPeaksDataDict['BW'].append(peak[2])
        PeakNum +=1
    except FileNotFoundError:
      failedIDs.append(IDs)
      continue
    except Exception as e:
      errorLog = (IDs, e)
      otherError.append(errorLog)
  else:
    redundantIDs.append(IDs)



In [ ]:
ProcessedPeaksDataDict['Age'] = []
ProcessedPeaksDataDict['Sex'] = []
for uniqueID in ProcessedPeaksDataDict['ID']:
    ProcessedPeaksDataDict['Age'].append(phenoDF.loc[uniqueID,'Age'])
    ProcessedPeaksDataDict['Sex'].append(phenoDF.loc[uniqueID,'Sex'])
print(len(ProcessedPeaksDataDict['Age']))
print(len(ProcessedPeaksDataDict['Sex']))

In [ ]:
processedDF = pd.DataFrame(ProcessedDataDict)
processedDF.to_csv('ProcessedAperData.csv', index = False)

In [ ]:

#multilevel data frame for peaks
ProcessedPeaksDF = pd.DataFrame(ProcessedPeaksDataDict)
ProcessedPeaksDF.sort_values(['ID', 'Peak_Number']).set_index(['ID', 'Peak_Number'])
ProcessedPeaksDF.to_csv('ProcessedPerData.csv', index = False)

# **The Following Section is to Run Null Hyphothesis Testing**

In [ ]:
import pandas as pd
import csv
import scipy
from scipy import stats

In [ ]:
%cd {EEG_Results_Directory}

In [ ]:
aperDF = pd.read_csv('ProcessedAperData.csv')
aperDF = aperDF.set_index('ID')

perDF = pd.read_csv('ProcessedPerData.csv') #not indexed by ID because there are multiple ID entries per person

## *Data Frame for test results*

In [ ]:
Test1 = [] #format 'independent var name v dependent var name'
AnovaStat = []
Anova_pValue = []

Test2 = []
LogRegression_coef = []
LogRegression_std_err = []
LogRegression_z = []
LogRegression_PZ = []

AnovaTable = pd.DataFrame({'Test Type' : Test1, 'F' : AnovaStat, 'P-Value' : Anova_pValue}).set_index('Test Type')
LogTable = pd.DataFrame({'Test Type' : Test2, 'coef' : LogRegression_coef, 'std Err' : LogRegression_std_err, 'z' : LogRegression_z, 'P > |z|' : LogRegression_PZ}).set_index('Test Type')




In [ ]:
male = 0
female = 0
for ID, row in aperDF.iterrows():
    if aperDF.loc[ID, "Sex"] == 0:
        female += 1
    elif aperDF.loc[ID, "Sex"] == 1:
        male += 1
    else:
        raise Exception('Unique Sex')

In [ ]:
female

In [ ]:
male

In [ ]:
female + male

## *The following tests treat age in groups 5 - 8, 9 - 12, and 13 - 22 rounding age to the nearest year and tested with a series of One Way Anovas. The later tests use a series of logistic regression tests to test sex related interactions. The final section used One way anovas to test subgroups in age to look for relations beneath the original groups* 

### **One Way Anovas**

#### *Test for Age V. Offset |      (H0) Offset is independent of Age      |      (H1) Offset varies based on an individual's Age* | α = 0.01

In [ ]:
#sort the data into age groups

youngest = []
middle = []
oldest = []

for ID, row in aperDF.iterrows():
        if (5 <= round(aperDF.loc[ID, 'Age']) <= 8):
            youngest.append(round(aperDF.loc[ID, 'Aper_offset'], 2))
        elif (9 <= round(aperDF.loc[ID, 'Age']) <= 12):
            middle.append(round(aperDF.loc[ID, 'Aper_offset'], 2))
        elif (13 <= round(aperDF.loc[ID, 'Age']) <= 22):
            oldest.append(round(aperDF.loc[ID, 'Aper_offset'], 2))
        else:   
            raise Exception('Participant Age is outside of expected range')

In [ ]:
print(len(youngest))
print(len(middle))
print(len(oldest))


In [ ]:
stat, pValue = scipy.stats.f_oneway(youngest, middle, oldest)

##### Result Entry

In [ ]:
AnovaTable.loc['Age v Aperiodic Offset', 'F']= stat
AnovaTable.loc['Age v Aperiodic Offset', 'P-Value'] = pValue

In [ ]:
AnovaTable  

#### *Test for Age V. Aperiodic Exponent |      (H0) Exponent is independent of Age      |      (H1) Exponent varies based on an individual's Age* | α = 0.01

In [ ]:
#sort the data into age groups

youngest = []
middle = []
oldest = []

for ID, row in aperDF.iterrows():
        if (5 <= round(aperDF.loc[ID, 'Age']) <= 8):
            youngest.append(round(aperDF.loc[ID, 'Aper_exp'], 2))
        elif (9 <= round(aperDF.loc[ID, 'Age']) <= 12):
            middle.append(round(aperDF.loc[ID, 'Aper_exp'], 2))
        elif (13 <= round(aperDF.loc[ID, 'Age']) <= 22):
            oldest.append(round(aperDF.loc[ID, 'Aper_exp'], 2))
        else:   
            raise Exception('Participant Age is outside of expected range')


In [ ]:
stat, pValue = scipy.stats.f_oneway(youngest, middle, oldest)

##### Result Entry

In [ ]:
AnovaTable.loc['Age v Aperiodic Exponent', 'F']= stat
AnovaTable.loc['Age v Aperiodic Exponent','P-Value'] = pValue

In [ ]:
AnovaTable

#### Test for Age V. Center Frequency | (H0) Center Frequency is independent of Age | (H1) Center Frequency varies based on Age | α = 0.01

In [ ]:
#sort the data into age groups

youngest = []
middle = []
oldest = []

for idx, row in perDF.iterrows():
    if (5 <= round(perDF.loc[idx, 'Age']) <= 8):
        youngest.append(perDF.loc[idx, 'CF'])
    elif (9 <= round(perDF.loc[idx, 'Age']) <= 12):
        middle.append(perDF.loc[idx, 'CF'])
    elif (13 <= round(perDF.loc[idx, 'Age']) <= 22):
        oldest.append(perDF.loc[idx, 'CF'])
    else:   
        raise Exception('Participant Age is outside of expected range')

In [ ]:
stat, pValue = scipy.stats.f_oneway(youngest, middle, oldest)

##### Result Entry

In [ ]:
AnovaTable.loc['Age v Center Frequency', 'F']= stat
AnovaTable.loc['Age v Center Frequency','P-Value'] = pValue

In [ ]:
AnovaTable

#### Test for Age V. Power | (H0) Power is independent of Age | (H1) Power varies based on Age | α = 0.01

In [ ]:
#sort the data into age groups

youngest = []
middle = []
oldest = []

for idx, row in perDF.iterrows():
    if (5 <= round(perDF.loc[idx, 'Age']) <= 8):
        youngest.append(perDF.loc[idx, 'PW'])
    elif (9 <= round(perDF.loc[idx, 'Age']) <= 12):
        middle.append(perDF.loc[idx, 'PW'])
    elif (13 <= round(perDF.loc[idx, 'Age']) <= 22):
        oldest.append(perDF.loc[idx, 'PW'])
    else:   
        raise Exception('Participant Age is outside of expected range')

In [ ]:
stat, pValue = scipy.stats.f_oneway(youngest, middle, oldest)

##### Result Entry

In [ ]:
AnovaTable.loc['Age v Power', 'F']= stat
AnovaTable.loc['Age v Power','P-Value'] = pValue

In [ ]:
AnovaTable

#### Test for Age V. Bandwidth | (H0) Bandwidth is independent of Age | (H1) Bandwidth varies based on Age | α = 0.001

In [ ]:
#sort the data into age groups

youngest = []
middle = []
oldest = []

for idx, row in perDF.iterrows():
    if (5 <= round(perDF.loc[idx, 'Age']) <= 8):
        youngest.append(perDF.loc[idx, 'BW'])
    elif (9 <= round(perDF.loc[idx, 'Age']) <= 12):
        middle.append(perDF.loc[idx, 'BW'])
    elif (13 <= round(perDF.loc[idx, 'Age']) <= 22):
        oldest.append(perDF.loc[idx, 'BW'])
    else:   
        raise Exception('Participant Age is outside of expected range')

In [ ]:
stat, pValue = scipy.stats.f_oneway(youngest, middle, oldest)

##### Result Entry

In [ ]:
AnovaTable.loc['Age v Bandwidth', 'F']= stat
AnovaTable.loc['Age v Bandwidth','P-Value'] = pValue

In [ ]:
AnovaTable

#### Test for Age V. Variance | (H0) Variance is independent of Age | (H1) Variance varies based on Age | α = 0.001 (modified due to the less precise nature of variance)

In [ ]:
#sort the data into age groups

youngest = []
middle = []
oldest = []

for ID, row in aperDF.iterrows():
        if (5 <= round(aperDF.loc[ID, 'Age']) <= 8):
            youngest.append(round(aperDF.loc[ID, 'Variance'], 2))
        elif (9 <= round(aperDF.loc[ID, 'Age']) <= 12):
            middle.append(round(aperDF.loc[ID, 'Variance'], 2))
        elif (13 <= round(aperDF.loc[ID, 'Age']) <= 22):
            oldest.append(round(aperDF.loc[ID, 'Variance'], 2))
        else:   
            raise Exception('Participant Age is outside of expected range')

In [ ]:
stat, pValue = scipy.stats.f_oneway(youngest, middle, oldest)

In [ ]:
AnovaTable.loc['Age v Bandwidth', 'F']= stat
AnovaTable.loc['Age v Bandwidth','P-Value'] = pValue

In [ ]:
AnovaTable

# **The Following Section is to run models and analyze results of the fitting**

In [ ]:
import numpy as np
import matplotlib
import mne
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa
import pandas as pd
import csv

import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.gam.api import GLMGam, BSplines
from statsmodels.graphics.api import abline_plot
import statsmodels.formula.api as smf
from pygam import LinearGAM, s, f
from loess.loess_2d import loess_2d

In [ ]:
aperDF = pd.read_csv('ProcessedAperData.csv')
aperDF = aperDF.set_index('ID')

perDF = pd.read_csv('ProcessedPerData.csv') #not indexed by ID because there are multiple ID entries per person

## Age and Sex Offset Model Regression Model (OLS)

The Plot for this Aperiodic Offset 

In [ ]:
data = aperDF.drop(['Aper_exp','Variance','Error'], axis=1)
# Assuming you have the DataFrame named "data"
# Plot data points for males (Sex == 1) with blue color
plt.scatter(data[data['Sex'] == 1]['Age'], data[data['Sex'] == 1]['Aper_offset'], color='blue', label='Male')

# Plot data points for females (Sex == 0) with red color
plt.scatter(data[data['Sex'] == 0]['Age'], data[data['Sex'] == 0]['Aper_offset'], color='red', label='Female')

# Set labels and title
plt.xlabel('Age')
plt.ylabel('Aper_offset')
plt.title('Scatter Plot of Aper_offset vs. Age (Color-coded by Sex)')

# Add a legend to distinguish between male and female data points
plt.legend()

# Show the plot
plt.show()


#### Youngest Group (5 - 8)

##### Get the features/dependent variable for this age range then create the Test and Traing data (75% training | 25% testing | Seed 4)

In [ ]:
x = aperDF.drop(['Aper_offset', 'Aper_exp', 'Variance', 'Error'], axis=1) #before seperating into the age group
y = aperDF['Aper_offset']

In [ ]:
#seperate X DF for the age group
IDs_to_drop = []
for ID, row in aperDF.iterrows():
    if round(aperDF.loc[ID, 'Age']) >= 9:
        IDs_to_drop.append(ID)
x = x.drop(IDs_to_drop, axis=0)
#seperate Y DF for the age group
y = y.drop(IDs_to_drop, axis= 0)


In [ ]:
training_x, testing_x, training_y, testing_y = train_test_split(x, y, test_size=0.25, random_state=4)

##### Fit OLS model using Age and Sex as features to predict Aperiodic Offset then analyze the results

In [ ]:
X_with_intercept = sm.add_constant(training_x)
model_stats_youngest = sm.OLS(training_y, X_with_intercept).fit()
print(model_stats_youngest.summary())

predictions = model_stats_youngest.predict(sm.add_constant(testing_x))
absolute_err = np.abs(testing_y - predictions).mean()
mean_sq_err = np.square(testing_y - predictions).mean()

print('==============================================================================\n'+'absolute error : ', absolute_err)
print('mean_squared_error : ', mean_sq_err)
print('______________________________________________________________________')

In [ ]:
dependent_variable_column = 'Aper_offset'
y.describe()

Note the R - Squared, P-Values, Residual plot, and Errors compared to the standard deviation and spread of the dataset. The linear model/the current features are poor predictors, likely underfitting the data. Given the known relation of age and sex as predictors of Aperiodic components and certain research suggesting non linear trends, another model is appropriate. This relation is even clearer when looking at the plots.

#### Plotting

In [ ]:
plt.scatter(training_x["Age"], training_y, color='orange', label='Training Data')
plt.scatter(testing_x["Age"], testing_y, color='blue', label='Testing Data')

sorted_indices = np.argsort(testing_x["Age"])
sorted_testing_x = testing_x.iloc[sorted_indices]
sorted_predictions = predictions[sorted_indices]

plt.xlabel('Age')
plt.ylabel('Aperiodic Offset')
plt.title('Aperiodic Offset Regression Model (5 - 8 years old)')
plt.plot(sorted_testing_x["Age"], sorted_predictions, color='k', label='Regression Line')

plt.legend()
plt.show()


In [ ]:
residuals = testing_y - predictions

plt.scatter(testing_x["Age"], residuals, color='green')
plt.xlabel('Predicted Aperiodic Offset')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.axhline(y=0, color='k', linestyle='--') 
plt.show()


### Middle Group (9 - 12)

#### Get the features/dependent variable for this age range then create the Test and Traing data (75% training | 25% testing | Seed 4)

In [ ]:
x = aperDF.drop(['Aper_offset', 'Aper_exp', 'Variance', 'Error'], axis=1) #before seperating into the age group
y = aperDF['Aper_offset']

In [ ]:
#seperate X DF for the age group
IDs_to_drop = []
for ID, row in aperDF.iterrows():
    if round(aperDF.loc[ID, 'Age']) <= 8 or round(aperDF.loc[ID, 'Age']) >= 13:
        IDs_to_drop.append(ID)
x = x.drop(IDs_to_drop, axis=0)
#seperate Y DF for the age group
y = y.drop(IDs_to_drop, axis= 0)


In [ ]:
training_x, testing_x, training_y, testing_y = train_test_split(x, y, test_size=0.25, random_state=4)

#### Fit OLS model using Age and Sex as features to predict Aperiodic Offset then analyze the results

Note the R - Squared, P-Values, Residual plot, and Errors compared to the standard deviation and spread of the dataset. The model is slightly better in this larger age range and age is a stronger indicater, however, errors compared to the range and standard deviation indicate there is some additional features or non linear relations to be considered. 

In [ ]:
X_with_intercept = sm.add_constant(training_x)
model_stats_middle = sm.OLS(training_y, X_with_intercept).fit()
print(model_stats_middle.summary())

predictions = model_stats_middle.predict(sm.add_constant(testing_x))
absolute_err = np.abs(testing_y - predictions).mean()
mean_sq_err = np.square(testing_y - predictions).mean()

print('==============================================================================\n'+'absolute error : ', absolute_err)
print('mean_squared_error : ', mean_sq_err)
print('______________________________________________________________________')

#### Plotting

In [ ]:
plt.scatter(training_x["Age"], training_y, color='orange', label='Training Data')
plt.scatter(testing_x["Age"], testing_y, color='blue', label='Testing Data')

sorted_indices = np.argsort(testing_x["Age"])
sorted_testing_x = testing_x.iloc[sorted_indices]
sorted_predictions = predictions[sorted_indices]

plt.xlabel('Age')
plt.ylabel('Aperiodic Offset')
plt.title('Aperiodic Offset Regression Model (9 - 12 years old)')
plt.plot(sorted_testing_x["Age"], sorted_predictions, color='k', label='Regression Line')

plt.legend()
plt.show()


In [ ]:
residuals = testing_y - predictions

plt.scatter(testing_x["Age"], residuals, color='green')
plt.xlabel('Predicted Aperiodic Offset')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.axhline(y=0, color='k', linestyle='--') 
plt.show()


### Oldest Group (13 - 22)

##### Get the features/dependent variable for this age range then create the Test and Traing data (75% training | 25% testing | Seed 4)

In [ ]:
x = aperDF.drop(['Aper_offset', 'Aper_exp', 'Variance', 'Error'], axis=1) #before seperating into the age group
y = aperDF['Aper_offset']

In [ ]:
#seperate X DF for the age group
IDs_to_drop = []
for ID, row in aperDF.iterrows():
    if round(aperDF.loc[ID, 'Age']) <= 12:
        IDs_to_drop.append(ID)
x = x.drop(IDs_to_drop, axis=0)
#seperate Y DF for the age group
y = y.drop(IDs_to_drop, axis= 0)


In [ ]:
training_x, testing_x, training_y, testing_y = train_test_split(x, y, test_size=0.25, random_state=4)

#### Fit OLS model using Age and Sex as features to predict Aperiodic Offset then analyze the results

In [ ]:
X_with_intercept = sm.add_constant(training_x)
model_stats_oldest = sm.OLS(training_y, X_with_intercept).fit()
print(model_stats_oldest.summary())

predictions = model_stats_oldest.predict(sm.add_constant(testing_x))
absolute_err = np.abs(testing_y - predictions).mean()
mean_sq_err = np.square(testing_y - predictions).mean()

print('==============================================================================\n'+'absolute error : ', absolute_err)
print('mean_squared_error : ', mean_sq_err)
print('______________________________________________________________________')

In [ ]:
dependent_variable_column = 'Aper_offset'
y.describe()

#### Plotting

In [ ]:
plt.scatter(training_x["Age"].tolist(), training_y, color='orange')

sorted_indices = np.argsort(testing_x["Age"])
sorted_testing_x = testing_x.iloc[sorted_indices]
sorted_predictions = predictions[sorted_indices]

plt.xlabel('Age')
plt.ylabel('Aperiodic Offset')
plt.title('Aperiodic Offset Linear Regression Model (13 - 22 years old)')
plt.plot(sorted_testing_x["Age"], sorted_predictions, color='k')

In [ ]:
residuals = testing_y - predictions

plt.scatter(testing_x["Age"], residuals, color='green')
plt.xlabel('Predicted Aperiodic Offset')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.axhline(y=0, color='k', linestyle='--') 
plt.show()


### **Full Model**

#### Get the features/dependent variable for this age range then create the Test and Traing data (75% training | 25% testing | Seed 4)

In [ ]:
x = aperDF.drop(['Aper_offset', 'Aper_exp', 'Variance', 'Error'], axis=1) #before seperating into the age group
y = aperDF['Aper_exp']

In [ ]:
training_x, testing_x, training_y, testing_y = train_test_split(x, y, test_size=0.25, random_state=4)

#### Fit OLS model using Age and Sex as features to predict Aperiodic Offset then analyze the results

Note the low P values and the standard errors. There is a negative linear trend but the stratified approach shows the potential non linear relations within

In [ ]:
X_with_intercept = sm.add_constant(training_x)
model_stats_oldest = sm.OLS(training_y, X_with_intercept).fit()
print(model_stats_oldest.summary())

predictions = model_stats_oldest.predict(sm.add_constant(testing_x))
absolute_err = np.abs(testing_y - predictions).mean()
mean_sq_err = np.square(testing_y - predictions).mean()

print('==============================================================================\n'+'absolute error : ', absolute_err)
print('mean_squared_error : ', mean_sq_err)
print('______________________________________________________________________')

In [ ]:
dependent_variable_column = 'Aper_offset'
y.describe()

In [ ]:
plt.scatter(training_x["Age"].tolist(), training_y, color='orange')

sorted_indices = np.argsort(testing_x["Age"])
sorted_testing_x = testing_x.iloc[sorted_indices]
sorted_predictions = predictions[sorted_indices]

plt.xlabel('Age')
plt.ylabel('Aperiodic Offset')
plt.title('Aperiodic Offset Linear Regression Model (5 - 22 years old)')
plt.plot(sorted_testing_x["Age"], sorted_predictions, color='k')

In [ ]:
residuals = testing_y - predictions

plt.scatter(testing_x["Age"], residuals, color='green')
plt.xlabel('Predicted Aperiodic Offset')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.axhline(y=0, color='k', linestyle='--') 
plt.show()


## Age and Sex Exponent Model Regression Model (OLS)

### Youngest Group (5 - 8)

In [ ]:
x = aperDF.drop(['Aper_offset', 'Aper_exp', 'Variance', 'Error'], axis=1) #before seperating into the age group
y = aperDF['Aper_exp']

In [ ]:
#seperate X DF for the age group
IDs_to_drop = []
for ID, row in aperDF.iterrows():
    if round(aperDF.loc[ID, 'Age']) >= 9:
        IDs_to_drop.append(ID)
x = x.drop(IDs_to_drop, axis=0)
#seperate Y DF for the age group
y = y.drop(IDs_to_drop, axis= 0)


In [ ]:
training_x, testing_x, training_y, testing_y = train_test_split(x, y, test_size=0.25, random_state=4)

#### Fit OLS model using Age and Sex as features to predict Aperiodic Offset then analyze the results

In [ ]:
X_with_intercept = sm.add_constant(training_x)
model_stats_youngest = sm.OLS(training_y, X_with_intercept).fit()
print(model_stats_youngest.summary())

predictions = model_stats_youngest.predict(sm.add_constant(testing_x))
absolute_err = np.abs(testing_y - predictions).mean()
mean_sq_err = np.square(testing_y - predictions).mean()

print('==============================================================================\n'+'absolute error : ', absolute_err)
print('mean_squared_error : ', mean_sq_err)
print('______________________________________________________________________')

In [ ]:
dependent_variable_column = 'Aper_offset'
y.describe()

#### Plotting

In [ ]:
plt.scatter(training_x["Age"], training_y, color='orange', label='Training Data')
plt.scatter(testing_x["Age"], testing_y, color='blue', label='Testing Data')

sorted_indices = np.argsort(testing_x["Age"])
sorted_testing_x = testing_x.iloc[sorted_indices]
sorted_predictions = predictions[sorted_indices]

plt.xlabel('Age')
plt.ylabel('Aperiodic Offset')
plt.title('Aperiodic Offset Regression Model (5 - 8 years old)')
plt.plot(sorted_testing_x["Age"], sorted_predictions, color='k', label='Regression Line')

plt.legend()
plt.show()


In [ ]:
residuals = testing_y - predictions

plt.scatter(testing_x["Age"], residuals, color='green')
plt.xlabel('Predicted Aperiodic Offset')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.axhline(y=0, color='k', linestyle='--') 
plt.show()


### Middle Group (9 - 12)


#### Get the features/dependent variable for this age range then create the Test and Traing data (75% training | 25% testing | Seed 4)

In [ ]:
x = aperDF.drop(['Aper_offset', 'Aper_exp', 'Variance', 'Error'], axis=1) #before seperating into the age group
y = aperDF['Aper_exp']

In [ ]:
#seperate X DF for the age group
IDs_to_drop = []
for ID, row in aperDF.iterrows():
    if round(aperDF.loc[ID, 'Age']) <= 8 or round(aperDF.loc[ID, 'Age']) >= 13:
        IDs_to_drop.append(ID)
x = x.drop(IDs_to_drop, axis=0)
#seperate Y DF for the age group
y = y.drop(IDs_to_drop, axis= 0)


In [ ]:
training_x, testing_x, training_y, testing_y = train_test_split(x, y, test_size=0.25, random_state=4)

#### Fit OLS model using Age and Sex as features to predict Aperiodic Offset then analyze the results

In [ ]:
X_with_intercept = sm.add_constant(training_x)
model_stats_middle = sm.OLS(training_y, X_with_intercept).fit()
print(model_stats_middle.summary())

predictions = model_stats_middle.predict(sm.add_constant(testing_x))
absolute_err = np.abs(testing_y - predictions).mean()
mean_sq_err = np.square(testing_y - predictions).mean()

print('==============================================================================\n'+'absolute error : ', absolute_err)
print('mean_squared_error : ', mean_sq_err)
print('______________________________________________________________________')

#### Plotting

In [ ]:
plt.scatter(training_x["Age"], training_y, color='orange', label='Training Data')
plt.scatter(testing_x["Age"], testing_y, color='blue', label='Testing Data')

sorted_indices = np.argsort(testing_x["Age"])
sorted_testing_x = testing_x.iloc[sorted_indices]
sorted_predictions = predictions[sorted_indices]

plt.xlabel('Age')
plt.ylabel('Aperiodic Offset')
plt.title('Aperiodic Offset Regression Model (9 - 12 years old)')
plt.plot(sorted_testing_x["Age"], sorted_predictions, color='k', label='Regression Line')

plt.legend()
plt.show()


In [ ]:
residuals = testing_y - predictions

plt.scatter(testing_x["Age"], residuals, color='green')
plt.xlabel('Predicted Aperiodic Offset')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.axhline(y=0, color='k', linestyle='--') 
plt.show()


### Oldest Group (13 - 22)

In [ ]:
x = aperDF.drop(['Aper_offset', 'Aper_exp', 'Variance', 'Error'], axis=1) #before seperating into the age group
y = aperDF['Aper_exp']

In [ ]:
#seperate X DF for the age group
IDs_to_drop = []
for ID, row in aperDF.iterrows():
    if round(aperDF.loc[ID, 'Age']) <= 12:
        IDs_to_drop.append(ID)
x = x.drop(IDs_to_drop, axis=0)
#seperate Y DF for the age group
y = y.drop(IDs_to_drop, axis= 0)


In [ ]:
training_x, testing_x, training_y, testing_y = train_test_split(x, y, test_size=0.25, random_state=4)

#### Fit OLS model using Age and Sex as features to predict Aperiodic Offset then analyze the results

In [ ]:
X_with_intercept = sm.add_constant(training_x)
model_stats_oldest = sm.OLS(training_y, X_with_intercept).fit()
print(model_stats_oldest.summary())

predictions = model_stats_oldest.predict(sm.add_constant(testing_x))
absolute_err = np.abs(testing_y - predictions).mean()
mean_sq_err = np.square(testing_y - predictions).mean()

print('==============================================================================\n'+'absolute error : ', absolute_err)
print('mean_squared_error : ', mean_sq_err)
print('______________________________________________________________________')

In [ ]:
dependent_variable_column = 'Aper_offset'
y.describe()

#### Plotting

In [ ]:
plt.scatter(training_x["Age"].tolist(), training_y, color='orange')

sorted_indices = np.argsort(testing_x["Age"])
sorted_testing_x = testing_x.iloc[sorted_indices]
sorted_predictions = predictions[sorted_indices]

plt.xlabel('Age')
plt.ylabel('Aperiodic Offset')
plt.title('Aperiodic Offset Linear Regression Model (13 - 22 years old)')
plt.plot(sorted_testing_x["Age"], sorted_predictions, color='k')

In [ ]:
residuals = testing_y - predictions

plt.scatter(testing_x["Age"], residuals, color='green')
plt.xlabel('Predicted Aperiodic Offset')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.axhline(y=0, color='k', linestyle='--') 
plt.show()


### Non linear modeling for youngest group Age and Sex Offset

In [ ]:
x = aperDF.drop(['Aper_offset', 'Aper_exp', 'Variance', 'Error'], axis=1) #before seperating into the age group
y = aperDF['Aper_offset']

In [ ]:
#seperate X DF for the age group
IDs_to_drop = []
for ID, row in aperDF.iterrows():
    if round(aperDF.loc[ID, 'Age']) >= 9:
        IDs_to_drop.append(ID)
x = x.drop(IDs_to_drop, axis=0)
#seperate Y DF for the age group
y = y.drop(IDs_to_drop, axis= 0)


In [ ]:
x_array = x.to_numpy()

In [ ]:
gam = LinearGAM(s(0) + f(1))
gam.gridsearch(x_array, y)
gam.summary()

In [ ]:
plt.figure()
fig, ax = plt.subplots()
XX = gam.generate_X_grid(term=0)
ax.plot(XX[:, 0], gam.partial_dependence(term=0, X=XX))
ax.plot(XX[:, 0], gam.partial_dependence(term=0, X=XX, width=0.95)[1], c='r', ls='--')
plt.xlabel('Age')
plt.show()

In [ ]:
y_pred = gam.predict(x_array)

plt.scatter(x_array[:, 0], y, label='Actual')
plt.plot(x_array[:, 0], y_pred, color='red', label='Predicted GAM Line')
plt.xlabel('Age')
plt.ylabel('Offset')
plt.legend()
plt.show()

### Non linear modeling for youngest group Age and Sex Exponent

In [ ]:
x = aperDF.drop(['Aper_offset', 'Aper_exp', 'Variance', 'Error'], axis=1) #before seperating into the age group
y = aperDF['Aper_exp']

In [ ]:
#seperate X DF for the age group
IDs_to_drop = []
for ID, row in aperDF.iterrows():
    if round(aperDF.loc[ID, 'Age']) >= 9:
        IDs_to_drop.append(ID)
x = x.drop(IDs_to_drop, axis=0)
#seperate Y DF for the age group
y = y.drop(IDs_to_drop, axis= 0)


In [ ]:
x_array = x.to_numpy()

In [ ]:
gam = LinearGAM(s(0) + f(1))
gam.gridsearch(x_array, y)
gam.summary()

#### Plotting

In [ ]:
XX = gam.generate_X_grid(term=0)

In [ ]:
plt.figure()
fig, ax = plt.subplots()
XX = gam.generate_X_grid(term=0)
ax.plot(XX[:, 0], gam.partial_dependence(term=0, X=XX))
ax.plot(XX[:, 0], gam.partial_dependence(term=0, X=XX, width=0.95)[1], c='r', ls='--')
plt.xlabel('Age')
plt.show()

In [ ]:
y_pred = gam.predict(x_array)

plt.scatter(x_array[:, 0], y, label='Actual')
plt.plot(x_array[:, 0], y_pred, color='red', label='Predicted GAM Line')
plt.xlabel('Age')
plt.ylabel('Exponent')
plt.legend()
plt.show()

## Comparing Variance & Error across age

In [ ]:
VarianceDF = aperDF.drop(['Aper_offset', 'Aper_exp', 'Error'], axis=1)

In [ ]:
VarianceDF.sort_values('Age', ascending=True)

In [ ]:
VarianceX = aperDF.drop(['Aper_offset', 'Aper_exp', 'Error', 'Variance'], axis=1)
VarianceY = VarianceDF['Variance']

alpa of .01 because unprocessed and highly significant clear difference must be apparent

In [ ]:
training_x, testing_x, training_y, testing_y = train_test_split(VarianceX, VarianceY, test_size=0.25, random_state=4)

In [ ]:
X_with_intercept = sm.add_constant(training_x)
model_stats_middle = sm.OLS(training_y, X_with_intercept).fit()
print(model_stats_middle.summary())

predictions = model_stats_middle.predict(sm.add_constant(testing_x))
absolute_err = np.abs(testing_y - predictions).mean()
mean_sq_err = np.square(testing_y - predictions).mean()

print('==============================================================================\n'+'absolute error : ', absolute_err)
print('mean_squared_error : ', mean_sq_err)
print('______________________________________________________________________')

In [ ]:
plt.scatter(VarianceDF['Age'].tolist(),VarianceDF['Variance'].tolist())

In [ ]:
ErrorDF = aperDF.drop(['Sex','Aper_offset', 'Aper_exp', 'Variance'], axis=1)

In [ ]:
ErrorDF.sort_values('Age', ascending=True)

In [ ]:
plt.scatter(ErrorDF['Age'].tolist(), ErrorDF['Error'].tolist())

In [ ]:
#sort the data into age groups

youngest = []
middle = []
oldest = []

for ID, row in aperDF.iterrows():
        if (5 <= round(aperDF.loc[ID, 'Age']) <= 8):
            youngest.append(round(aperDF.loc[ID, 'Error'], 2))
        elif (9 <= round(aperDF.loc[ID, 'Age']) <= 12):
            middle.append(round(aperDF.loc[ID, 'Error'], 2))
        elif (13 <= round(aperDF.loc[ID, 'Age']) <= 22):
            oldest.append(round(aperDF.loc[ID, 'Error'], 2))
        else:   
            raise Exception('Participant Age is outside of expected range')


## Final Tests: Interaction Age + Sex offset and exponent, Stratified sex for both

In [ ]:
aperDF

## Interaction Testing

In [ ]:
model = smf.ols(formula='Aper_offset ~ Age + Sex + Age:Sex', data= aperDF).fit()
summary = model.summary()
summary.tables[1]

In [ ]:
aperDF['Sex_med'] = aperDF.Sex > aperDF.Sex.median()
aperDF['Sex_med'] = np.where(aperDF.Sex_med == False, 'Below Median', 'Above Median')
sns.lmplot(x='Age', y='Aper_offset', hue='Sex', data=aperDF, scatter_kws={'alpha':0.3},ci=None)
plt.ylabel("Aperiod Offset")
plt.xlabel("Age")

In [ ]:
model = smf.ols(formula='Aper_exp ~ Age + Sex + Age:Sex', data= aperDF).fit()
summary = model.summary()
summary.tables[1]

In [ ]:
aperDF['Sex_med'] = aperDF.Sex > aperDF.Sex.median()
aperDF['Sex_med'] = np.where(aperDF.Sex_med == False, 'Below Median', 'Above Median')
sns.lmplot(x='Age', y='Aper_exp', hue='Sex', data=aperDF, scatter_kws={'alpha':0.3},ci=None)
plt.ylabel("Aperiod Exponent")
plt.xlabel("Age")